In [ ]:
# !wget -O test.mcool https://osf.io/3h9js/download	

In [ ]:
%env ANYWIDGET_HMR=1

In [ ]:
import higlass as hg
import ipywidgets
import traitlets

import itertools

# TODO: Need to find an abstraction that brings this into notebooks.
class Tilesets(ipywidgets.DOMWidget):
    value = traitlets.Int(0).tag(sync=True)
    def __init__(self):
        super().__init__()
        self.on_msg(self._handle_custom_msg)
        self.ts = dict()

    def add(self, ts):
        self.ts[ts.tileset.uid] = ts.tileset
        return self

    def _handle_custom_msg(self, msg, buffers):
        match msg["payload"]:
            case { "type": "tileset_info", "tilesetUid": tileset_uid }:
                self.send({
                    "id": msg["id"],
                    "payload": { tileset_uid: self.ts[tileset_uid].info() },
                })
            case { "type": "tiles", "tileIds": tile_ids }:
                tiles = []
                for tileset_uid, group_tile_ids in itertools.groupby(
                    iterable=sorted(tile_ids), key=lambda tile_id: tile_id.split(".")[0]
                ):
                    tiles.extend(
                        self.ts[tileset_uid].tiles(list(group_tile_ids))
                    )
                self.send({
                    "id": msg["id"],
                    "payload": { tile_id: tile for tile_id, tile in tiles },
                })
            case _:
                raise ValueError("Something's wrong with the Internet")

ts = hg.cooler("./test.mcool")
tss = Tilesets().add(ts)
track = ts.track("heatmap")
hg.view(track, width=6).widget(ts=tss)